### Loading in the data



In [325]:
import matplotlib.pyplot as plt
#import statsmodels.tsa.seasonal as smt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import datetime as dt

In [326]:
import os
from os.path import expanduser
home = expanduser("~")
os.chdir(home+'\Desktop\Deep Learning Project\Data\price-volume-data-for-all-us-stocks-etfs\Data\Stocks')
os.getcwd()

'C:\\Users\\Thomas\\Desktop\\Deep Learning Project\\Data\\price-volume-data-for-all-us-stocks-etfs\\Data\\Stocks'

In [908]:
#read data
#takes 1 minute for 1000 files

filenames = [x for x in os.listdir() if x.endswith('.txt') and os.path.getsize(x) > 0]
#filenames = ['a.us.txt', aa.us.txt, 'bgr.us.txt']
filenames=filenames[:100]


#data = []
result=None
for filename in filenames:
    df = pd.read_csv(filename, sep=',')

    label, _, _ = filename.split(sep='.')
    df.insert(loc=0, column='Label', value=filename)
    df['Date'] = pd.to_datetime(df['Date'])
    df['ReturnOpenPrevious']=(df.Open-df.Open.shift(1))/df.Open.shift(1)
    df['ReturnOpenNext']=(df.Open.shift(-1)-df.Open)/df.Open
    #data.append(df)
    if type(result)==type(None):
        result=df
    else:
        result=result.append(df,ignore_index=True)
    
result.head()

,Label,Date,Open,High,Low,Close,Volume,OpenInt,ReturnOpenPrevious,ReturnOpenNext
0,a.us.txt,1999-11-18,30.713,33.754,27.002,29.702,66277506,0,NaN,-0.056230
1,a.us.txt,1999-11-19,28.986,29.027,26.872,27.257,16142920,0,-0.056230,-0.037949
2,a.us.txt,1999-11-22,27.886,29.702,27.044,29.702,6970266,0,-0.037949,0.028760
3,a.us.txt,1999-11-23,28.688,29.446,27.002,27.002,6332082,0,0.028760,-0.055947
4,a.us.txt,1999-11-24,27.083,28.309,27.002,27.717,5132147,0,-0.055947,0.018868


In [910]:
result1=result.copy()
encoder_Label = dict()
encoder_Label = {l: id for id, l in enumerate(filenames)}

result1.Label=[encoder_Label.get(i) for i in result1.Label]
result1.tail()

,Label,Date,Open,High,Low,Close,Volume,OpenInt,ReturnOpenPrevious,ReturnOpenNext
260881,99,2017-11-06,8.00,8.15,7.800,7.85,409212,0,0.059603,-0.012500
260882,99,2017-11-07,7.90,8.00,7.750,7.80,464754,0,-0.012500,-0.018987
260883,99,2017-11-08,7.75,7.90,7.525,7.80,392643,0,-0.018987,-0.006452
260884,99,2017-11-09,7.70,7.90,7.500,7.85,465090,0,-0.006452,0.025974
260885,99,2017-11-10,7.90,8.15,7.750,7.95,558278,0,0.025974,NaN


In [911]:
result2=result1.copy()
encoder_Date = dict()
encoder_Date = {l: id for id, l in enumerate(result2.Date.unique())}
result2.Date=[encoder_Date.get(i) for i in result2.Date.values]
result2.tail()

,Label,Date,Open,High,Low,Close,Volume,OpenInt,ReturnOpenPrevious,ReturnOpenNext
260881,99,4516,8.00,8.15,7.800,7.85,409212,0,0.059603,-0.012500
260882,99,4517,7.90,8.00,7.750,7.80,464754,0,-0.012500,-0.018987
260883,99,4518,7.75,7.90,7.525,7.80,392643,0,-0.018987,-0.006452
260884,99,4519,7.70,7.90,7.500,7.85,465090,0,-0.006452,0.025974
260885,99,4520,7.90,8.15,7.750,7.95,558278,0,0.025974,NaN


8

In [914]:
#result3=np.empty((len(encoder_Label),len(encoder_Date),6))
#result3[:,:,:]=None
#result3.shape
#for index, row in result2.iterrows():
#   result3[int(row['Label']),int(row['Date']),:]=row[['Open','High','Low','Close','Volume','OpenInt']]
#result3

result3=np.empty((len(encoder_Label),len(encoder_Date),result2.shape[1]-2))
result3[:,:,:]=None

for row in result2.itertuples():
    result3[int(row.Label),int(row.Date),:]=row[3:]
result3[0,1,:]


array([ 2.89860000e+01,  2.90270000e+01,  2.68720000e+01,  2.72570000e+01,
        1.61429200e+07,  0.00000000e+00, -5.62302608e-02, -3.79493549e-02])

In [792]:
result3.shape

(1000, 12077, 6)

### Baseline Model
### Starting from result

In [915]:
result


,Label,Date,Open,High,Low,Close,Volume,OpenInt,ReturnOpenPrevious,ReturnOpenNext
0,a.us.txt,1999-11-18,30.713,33.754,27.002,29.702,66277506,0,NaN,-0.056230
1,a.us.txt,1999-11-19,28.986,29.027,26.872,27.257,16142920,0,-0.056230,-0.037949
2,a.us.txt,1999-11-22,27.886,29.702,27.044,29.702,6970266,0,-0.037949,0.028760
3,a.us.txt,1999-11-23,28.688,29.446,27.002,27.002,6332082,0,0.028760,-0.055947
4,a.us.txt,1999-11-24,27.083,28.309,27.002,27.717,5132147,0,-0.055947,0.018868
5,a.us.txt,1999-11-26,27.594,28.012,27.509,27.807,1832635,0,0.018868,0.002972
6,a.us.txt,1999-11-29,27.676,28.650,27.380,28.432,4317826,0,0.002972,0.024353
7,a.us.txt,1999-11-30,28.350,28.986,27.634,28.480,4567146,0,0.024353,0.004586
8,a.us.txt,1999-12-01,28.480,29.324,28.273,28.986,3133746,0,0.004586,0.036938
9,a.us.txt,1999-12-02,29.532,30.375,29.155,29.786,3252997,0,0.036938,0.027225
